#Extracción de características y entrenamiento de modelos

###POr un lado, en este notebook vamos a realizar la extracciòn de caracteríticas de dos formas diferentes, mediante CountVectorizer y TfidfVectorizer, posteriormente realizaremos una seleccion de características con SelectKBest(chi2) y realizaremos el entrenamiento de los modelos con los dos set de datos con características diferentes. Es decir vamos a generar dos pipeline diferentes y veremos como se comporta en el entrenamiento.
**Pipelines:**

TfidfVectorizer -> SelectKBest(chi2) -> LogisticRegression

CountVectorizer -> SelectKBest(chi2) -> LogisticRegression

TfidfVectorizer -> SelectKBest(chi2) -> GradientBoostingClassifier

CountVectorizer -> SelectKBest(chi2) -> GradientBoostingClassifier

###Por otro lado vamos a entrenar varios modelos de deep learning. Primeramente entrenaremos 3 modelos con el Embedding realizado en la propia red y probaremos con tres capas diferentes  ['LSTM','GRU','SimpleRNN']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

###En la siguiente linea de codigo cargamos los diccionario con los sets de que necesitamos para el entrenamiento. Por un lado dic_training_sets con el conjunto de train y test para entrenar el modelo de regresión logística y boosting y por otro lado el dic_cnn_sets que contiene los datos para realizar la cnn con el embedding en una capa y que tambien contiene la embedding layer generada con word2vec en el notebook preprocesing

In [ ]:
import pickle
path = '/content/drive/MyDrive/NLP_practica/datasets/' #path en donde esta almacenado el df

# carga el diccionario desde el archivo
with open(f'{path}dic_training_sets.pkl', 'rb') as archivo:
    dic_training_sets = pickle.load(archivo)
    
with open(f'{path}dic_cnn_sets.pkl', 'rb') as archivo:
    dic_cnn_sets = pickle.load(archivo)
    

In [ ]:
import codecs
import os
import pandas as pd
import numpy as np
import string
import random
import pickle

import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.probability import FreqDist

from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GRUV2, SimpleRNN

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV # Modelado
from sklearn.pipeline import Pipeline # Modelado
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer # Modelado
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.linear_model import LogisticRegression # Reporte
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_curve, precision_recall_curve

path = '/content/drive/MyDrive/NLP_practica/datasets/' #path en donde esta almacenado el df

###Vamos a generar una clase que se encarga de todo el proceso de forma automática. Es decir, podremos indicarle el tipo de extraccion(tfidf o countvectorizer) y el modelo a entrenar(regresion o GradientBoostingClassifier).
La clase automaticamente carga los hyperparámetros a validar y sus rangos, es posible realizar modificaciones de dichos rangos pasando dicho hyperparametro como argumento con el nuevo rango a validar al instanciar la clase. Por lo tanto automaticamente genera el pipeline solicitado, hace validación cruzada de parámetros y entrena la mejor combinación, devolviendo los resultados. Durante el proceso ira generando info sobre el paso que está realizando

In [ ]:
class TextPipelineLogisticAndBoos():
  """Esta clase genera el pipeline indicado realizando la extracción de características, calcula los mejores hyperparametros, entrena el modelo con dos posibles algoritmos logistic regresion y 
  GradientBoostingClassifier con dichos parametros y realiza las predicciones.  A lo largo del proceso se iran mostrando en pantalla información del paso que se está realizando asi como algun dato de interes"""

  def __init__(self, method_extraction:str = 'tfidf', model:str= 'regresion',tfidf__ngram_range: list= [(1, 1), (1, 2)], tfidf__min_df: list = [1, 2],
               tfidf__max_df:list = [0.9, 0.95],tfidf__max_features:list = [1000,2500], 
               chi2__k:list = [10, 100], logistic__C:list = [0.1, 1.0],
               n_estimators:list = [50, 100], max_depth:list= [3, 5], learning_rate:list = [0.01,0.1]) -> None:
    """el constructor de la clase crea las variables de clase que contienen el metodo de extracción y el modelo a entrenar. Luego llama a funciones para crear el diccionario de hiperparámetros con los 
    rangos a validar(establecidos por defecto), si se quieren modificar se pueden pasar como argumento al instanciar la clase ,y el pipeline elgido guardandolo en variables de clase.
      :param method_extraction: metodo por defecto tfidf, opcion 2: 'countVectorizer
      :param model: modelo por defecto regresion, opcion 2: 'boosting
      :params hyper:tfidf__ngram_range: list= [(1, 1), (1, 2)], tfidf__min_df: list = [1, 2],
               tfidf__max_df:list = [0.9, 0.95],tfidf__max_features:list = [1000,2500], 
               chi2__k:list = [10, 100], logistic__C:list = [0.1, 1.0],
               n_estimators:list = [50, 100], max_depth:list= [3, 5], learning_rate:list = [0.01,0.1] """
    self.method_extraction = method_extraction
    self.model = model
    self.params = self.createDictHyperparams(tfidf__ngram_range= tfidf__ngram_range, tfidf__min_df = tfidf__min_df,
               tfidf__max_df= tfidf__max_df,tfidf__max_features= tfidf__max_features, 
               chi2__k= chi2__k, logistic__C = logistic__C,
               n_estimators= n_estimators, max_depth= max_depth, learning_rate= learning_rate)
    self.pipe = self.pipelineMethod()
    
  def run(self,X_train: pd.Series, X_test: pd.Series, y_train: pd.Series, y_test: pd.Series) -> tuple:
    """este método ejecuta todo el pipeline indicado automaticamente, y va generando mensajes durante todo el proceso
      : param X_train: pd.Series  conjunto de train
      : param X_test: pd.Series  conjunto de test
      : param y_train: pd.Series  etiquetas de train
      : param y_test: pd.Series  etiquetas de test
      : param method_extraction:str = 'tfidf' parámetro que define el pipeline,  por defecto TfidfVectorizer. se puede pasar por parámetro 'countVectorizer
      return : tuple --metrics,train_predict, test_predict"""
    name_model = f'{self.method_extraction}_{self.model}'
    pipeline_steps = list(self.pipe.named_steps.keys())
    print('\n#################################################################################\n')
    print(f'Iniciamos el cross-validation del pipeline: {pipeline_steps[0]} ->  {pipeline_steps[1]} -> {pipeline_steps[2]}.')
    print('\n#################################################################################')
    print('\nHiperparámetros a validar en la cross-validation:')
    self.printDict(self.params)
    self.best_model = self.gridSearch(pipeline = self.pipe, params = self.params)
    model = self.fitGridModel(best_model = self.best_model, X_train= X_train, y_train= y_train)
    print('\nObtenemos los valores optimos para los hiperparámtros')
    print('<<<<<<<<<<<<<<<<<Best Hyperparamns>>>>>>>>>>>>>>>>>>>')
    self.printDict(model.best_params_)
    train_predict, test_predict = self.predictModel(model,X_train= X_train,X_test= X_test)
    print(f'\nMétricas obtenidas en el conjunto de train mediante {pipeline_steps[2]}')
    metrics_train = self.evaluateModel(y_train,train_predict, 'train')
    print(f'\nMétricas obtenidas en el conjunto de test mediante {pipeline_steps[2]}')
    metrics_test = self.evaluateModel(y_test,test_predict, 'test')
    metrics={'name_model':name_model}
    metrics.update(metrics_train)
    metrics.update(metrics_test)
    return metrics,train_predict, test_predict

    
  def createDictHyperparams(self,tfidf__ngram_range: list, tfidf__min_df: list,
               tfidf__max_df:list,tfidf__max_features:list, 
               chi2__k:list, logistic__C:list ,
               n_estimators:list, max_depth:list, learning_rate:list):
    """este metodo genera el diccionario de parametros para el grissearch """
    print('[INFO] Generando diccionario con hyperparametros a validar...')
    if self.method_extraction == 'tfidf':
      param_extract = {
      'tfidf__ngram_range':  tfidf__ngram_range,
      'tfidf__min_df': tfidf__min_df,
      'tfidf__max_df': tfidf__max_df,
      'tfidf__max_features': tfidf__max_features,
      'chi2__k': chi2__k,      
                        }
    if self.method_extraction == 'countVectorizer':
      param_extract = {
      'chi2__k': chi2__k
                     }

    if self.model == 'regresion':
      param_model = {
      'logistic__C': logistic__C
                      }
    if self.model == 'boosting':
      param_model = {
      'boosting__n_estimators': n_estimators,
      'boosting__max_depth': max_depth,
      'boosting__learning_rate': learning_rate
      }            
    return {**param_extract,**param_model}

  def pipelineMethod(self) -> Pipeline:
    """método que carga el pipeline dependiendo del método de extraccion y model pasado al instaciar la clase
      return : Pipeline"""
    print('[INFO] Generando el pipeline...')
    if self.method_extraction == 'tfidf':
      if self.model =='regresion':
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(ngram_range=(1, 1), min_df=1, max_df=1.0, max_features = 2500)),
            ('chi2', SelectKBest(chi2)),
            ('logistic', LogisticRegression(max_iter=1000))
        ])
      if self.model =='boosting':
        pipeline = Pipeline([
            ('tfidf', TfidfVectorizer(ngram_range=(1, 1), min_df=1, max_df=1.0, max_features = 2500)),
            ('chi2', SelectKBest(chi2)),
            ('boosting', GradientBoostingClassifier())
        ])
      return pipeline
    if self.method_extraction == 'countVectorizer':
      if self.model =='regresion':
        pipeline = Pipeline([
            ('countvectorizer',CountVectorizer()),
            ('chi2', SelectKBest(chi2)),
            ('logistic', LogisticRegression())
          ]) 
      if self.model =='boosting':
        pipeline = Pipeline([
            ('countvectorizer',CountVectorizer()),
            ('chi2', SelectKBest(chi2)),
            ('boosting', GradientBoostingClassifier())
          ])         
    return pipeline    

  def gridSearch(self, pipeline: Pipeline, params: dict, cv: int = 5) -> GridSearchCV:
    """metodo que carga el GridSearchCV
     : param pipeline: Pipeline 
     : param params: dict contien los hyperparámetros y los rangos a validar
     : param cv: int por defecto 5. número de validaciones
     return : objeto de la clase GridSearchCV"""
    print('\n[INFO] Creando el gridSeach ...')
    grid_search = GridSearchCV(pipeline, params, cv=cv, n_jobs=-1, verbose=1)    
    return grid_search

  def fitGridModel(self, best_model: GridSearchCV, X_train: pd.Series, y_train: pd.Series) -> GridSearchCV:
    """con este metedo se realiza el entrenamiento en base al modelo GridSearchCV pasado por parámetro
      : param best_model: GridSearchCV     
      : param X_train: pd.Series
      : param X_train: pd.Series
      return GridSearchCV entrenado """
    print('\n[INFO] Realizando el entrenamiento ...')
    best_model.fit(X_train, y_train)
    return best_model

  def predictModel(self, model, X_train: pd.Series,X_test: pd.Series) -> tuple:
    """con este metedo se realiza la prediccion del conjunto de train t test
      : param model: GridSearchCV     
      : param X_train: pd.Series
      : param X_train: pd.Series
      return tuple (train_predict, test_predict)"""
    print('\n[INFO] Realizando las predicciones del conjunto de train ...')
    train_predict = model.predict(X_train)
    print('[INFO] Realizando las predicciones del conjunto de test ...')
    test_predict = model.predict(X_test)
    return train_predict, test_predict

  @staticmethod
  def evaluateModel(y_true: pd.Series, y_pred: pd.Series, set:str) -> dict:
    """método que calcula e imprime por pantalla diferentes métricas
      : param y_true: etiquetas
      : param y_pred: predicciones
      return: dict con metricas"""
    acc = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = f1_score(y_true, y_pred)
    cm = confusion_matrix(y_true, y_pred)
    metrics = {
        f'accuracy_{set}': acc,
        f'precision_{set}': precision,
        f'recall_{set}': recall,
        f'f1_{set}' : f1,
        f'conf_matx_{set}': cm
    }
    print('Accuracy: {:.4f}'.format(acc))
    print('Precision: {:.4f}'.format(precision))
    print('Recall: {:.4f}'.format(recall))
    print('F1-Score: {:.4f}'.format(f1))
    print('Confusion Matrix:\n{}'.format(cm))
    return metrics

  def printDict(self,dic: dict) -> None:
    """metodo al que le pasas un diccionario e itera en los items para imprimirlos '{key}:    {value}'
     : param dic: dict
     return: None"""
    for key, value in dic.items():
      print(f'{key}:    {value}')




In [ ]:
#vamos a guardar en variables los diferentes set que hay en mi_diccionario
x_train = dic_training_sets['X_train']
y_train = dic_training_sets['y_train']
x_test = dic_training_sets['X_test']
y_test = dic_training_sets['y_test']

##Ahora vamos a crear las diferentes instancias para el entrenamiento de los diferentes modelos y extractores de características

##1.    TfidfVectorizer -> SelectKBest(chi2) -> LogisticRegression 

In [ ]:
#Creamos la instacia a la clase TextPipelineLogistic que se encarga del análisis
pipeline_tdidf_reg = TextPipelineLogisticAndBoos(method_extraction='tfidf', model='regresion')

In [ ]:
#llamamos al método run de la clase instanciada para que genere el pipeline con method_extraction='tfidf'
metrics_tfidf_reg,train_pred_tfidf_reg, test_pred_tfidf_reg = pipeline_tdidf_reg.run(X_train = x_train, X_test = x_test, y_train = y_train, y_test = y_test)

##2.    CountVectorizer -> SelectKBest(chi2) -> LogisticRegression

In [ ]:
pipeline_countV_reg = TextPipelineLogisticAndBoos(method_extraction='countVectorizer', model='regresion')

In [ ]:
#llamamos al método run de la clase instanciada para que genere el pipeline 
metrics_vec_reg,train_pred_vec_reg, test_pred_vec_reg = pipeline_countV_reg.run(X_train = x_train, X_test = x_test, y_train = y_train, y_test = y_test)

##3. TfidfVectorizer -> SelectKBest(chi2) -> GradientBoostingClassifier

In [ ]:
pipeline_tdidf_boos = TextPipelineLogisticAndBoos(method_extraction='tfidf', model='boosting')

In [ ]:
#llamamos al método run de la clase instanciada para que genere el pipeline 
metrics_tfidf_boos,train_pred_tfidf_boos, test_pred_tfidf_boos = pipeline_tdidf_boos.run(X_train = x_train, X_test = x_test, y_train = y_train, y_test = y_test)

####4. CountVectorizer -> SelectKBest(chi2) -> GradientBoostingClassifier

In [ ]:
pipeline_countV_boos = TextPipelineLogisticAndBoos(method_extraction='countVectorizer', model='boosting')

In [ ]:
#llamamos al método run de la clase instanciada para que genere el pipeline 
metrics_vec_boos,train_pred_vec_boos, test_pred_vec_boos = pipeline_countV_boos.run(X_train = x_train, X_test = x_test, y_train = y_train, y_test = y_test)

In [ ]:
metrics_models_reg_boos = pd.DataFrame(columns=['name_model','accuracy_train', 'precision_train', 'recall_train', 'f1_train',  'accuracy_test', 'precision_test', 'recall_test', 'f1_test'])

In [ ]:
metrics_models_reg_boos = metrics_models_reg_boos.append(metrics_vec_reg, ignore_index=True)
metrics_models_reg_boos = metrics_models_reg_boos.append(metrics_vec_boos, ignore_index=True)
metrics_models_reg_boos = metrics_models_reg_boos.append(metrics_tfidf_reg, ignore_index=True)
metrics_models_reg_boos = metrics_models_reg_boos.append(metrics_tfidf_boos, ignore_index=True)

In [ ]:
# Especificamos la ruta de la carpeta en donde se guardará, si la carpeta aun no existe la creará
path = '/content/drive/MyDrive/NLP_practica/datasets/'
metrics_models_reg_boos.to_csv(f'{path}metrics_models_reg_boos.csv', index=False)

metrics_models_reg_boos

#Modelo con Deep Learning

###Vamos a evaluar 3 modelos con el embedding realizado con la funcion de keras Embedding, probando con 3 capas diferentes 'LSTM' , 'GRU' y 'SimpleRNN'
Para ello vamos a crear una clase que realiza de forma automatica el modelado de los tres sistemas, devolviéndonos las diferntens métricas en pantalla y a su vez en un df

In [ ]:
padded_sequences_train = dic_cnn_sets['padded_sequences_train']
padded_sequences_test = dic_cnn_sets['padded_sequences_test']
y_train = dic_cnn_sets['y_train']
y_test = dic_cnn_sets['y_test']
embedding_layer = dic_cnn_sets['embedding_layer']

In [ ]:
class TrainingCNN:
  """esta clase realiza el entrenamiento con CNN con diferentes capas"""
  def __init__(self,padded_sequences_train,padded_sequences_test, y_train, y_test ,embedding_layer) -> None:
    """el constructor genera las varibles de clase que almacena los pad_sets los y´s y el embedding_layer
      :param padded_sequences_train: conjunto de train con padding
      :param padded_sequences_test: conjunto de test con padding
      :param y_train: etiquetas del train
      :param y_test: etiquetas del test"""
    self.padded_sequences_train= padded_sequences_train
    self.padded_sequences_test= padded_sequences_test
    self.y_train = y_train
    self.y_test = y_test
    self.embedding_layer = embedding_layer
    
  def run(self, input_dim: int=10882, output_dim: int=100, input_length: int=1200,n_neural: int = 100) -> pd.DataFrame:
    """metodo que ejecuta la creacion de los modelos, sus entrenamientos y almacena las metricas de los mismos en un df
      : param input_dim: int dimension de entrada 10882 
      : param output_dim: int dimension de salid 100
      : param  input_length: int tamaño máximo
      : param n_neural: numero de neuronas
      return: pd.DataFrame con las metricas de los 3 modelos"""
    #Definimos una variable que contiene los metodos de la capa profunda
    type_layers = ['LSTM','GRU','SimpleRNN']
    #Generamos un df para cada tipo de cnn entrenada(embedding en capa, Word2Vec) donde iremos almacenando las metricas obtenidas en los entrenamientos y predicciones
    metrics_cnn_df = pd.DataFrame(columns=['layer', 'loss_train', 'accuracy_train', 'precision_train', 'recall_train', 'f1_train', 'loss_test', 'accuracy_test', 'precision_test', 'recall_test', 'f1_test'])
    metrics_cnn_custom_df = pd.DataFrame(columns=['layer', 'loss_train', 'accuracy_train', 'precision_train', 'recall_train', 'f1_train', 'loss_test', 'accuracy_test', 'precision_test', 'recall_test', 'f1_test'])
    
    #Vamos a ejecutar los entrenamientos de forma automática con dos bucles, uno para cada modelo, que iran realando entrenamiento con las 3 type_layers
    #Primero el modelo con el embedding realizado en la capa de la CNN
    
    for layer in type_layers:
      print('\n#################################################################################')
      print(f'\n            Generando modelo con Embedding en capa profunda')
      self.model_cnn_embedding = self.modelCnnEmbedding(rnn_type = layer, input_dim=input_dim, output_dim=output_dim, input_length=input_length,n_neural= n_neural)
      print(f'\n[INFO]Entrenando modelo CNN con capa {layer}')
      self.model_cnn_embedding.fit(self.padded_sequences_train,self.y_train, validation_split=0.2,batch_size=64, epochs=1)
      print('\nGeneramos las métricas de train:')
      metrics_train = self.evaluate_model(self.model_cnn_embedding,self.padded_sequences_train,self.y_train)
      print('\nGeneramos las métricas de test:')
      metrics_test = self.evaluate_model(self.model_cnn_embedding,self.padded_sequences_test,self.y_test)
      metrics_dict = {'layer': layer,
                        'loss_train': metrics_train['loss'],
                        'accuracy_train': metrics_train['accuracy'],
                        'precision_train': metrics_train['precision'],
                        'recall_train': metrics_train['recall'],
                        'f1_train': metrics_train['f1'],
                        'loss_test': metrics_test['loss'],
                        'accuracy_test': metrics_test['accuracy'],
                        'precision_test': metrics_test['precision'],
                        'recall_test': metrics_test['recall'],
                        'f1_test':metrics_test['f1']
                       }
        
      metrics_cnn_df = metrics_cnn_df.append(metrics_dict, ignore_index=True)

    #realizamos el mismo proceso con la capa de embedding realizada con W2V
    for layer in type_layers:
      print('\n#################################################################################')
      print(f'\n            Generando modelo con Embedding customizado W2V')
      self.model_cnn_embedding = self.modelCnnW2VEmbedding(rnn_type = layer, n_neural = n_neural)
      print(f'\n[INFO]Entrenando modelo CNN con capa {layer}')
      self.model_cnn_embedding.fit(self.padded_sequences_train,self.y_train, validation_split=0.2,batch_size=64, epochs=1)
      print('\nGeneramos las métricas de train:')
      metrics_train = self.evaluate_model(self.model_cnn_embedding,self.padded_sequences_train,self.y_train)
      print('\nGeneramos las métricas de test:')
      metrics_test = self.evaluate_model(self.model_cnn_embedding,self.padded_sequences_test,self.y_test)
      metrics_dict = {'layer': layer,
                        'loss_train': metrics_train['loss'],
                        'accuracy_train': metrics_train['accuracy'],
                        'precision_train': metrics_train['precision'],
                        'recall_train': metrics_train['recall'],
                        'f1_train': metrics_train['f1'],
                        'loss_test': metrics_test['loss'],
                        'accuracy_test': metrics_test['accuracy'],
                        'precision_test': metrics_test['precision'],
                        'recall_test': metrics_test['recall'],
                        'f1_test':metrics_test['f1']
                       }
        
      metrics_cnn_custom_df = metrics_cnn_custom_df.append(metrics_dict, ignore_index=True)
    
    return  metrics_cnn_df, metrics_cnn_custom_df

  def modelCnnEmbedding(self, rnn_type: str, input_dim: int,output_dim:int , n_neural: int, input_length: int):
    """este metodo generea y compila un modelo cnn con el embedding automatico de keras Embedding
     :param rnn_type: tipo de capa. Opciones ['LSTM','GRU','SimpleRNN']
     return: model"""
    print(f'\n####################### Modelo con capa {rnn_type} ####################################\n')
    print(f'[INFO] Generando el modelo...')
    model = Sequential()
    model.add(Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length))
    if rnn_type == 'LSTM':
          model.add(LSTM(n_neural))
    elif rnn_type == 'GRU':
        model.add(GRUV2(n_neural))  
    elif rnn_type == 'SimpleRNN':
        model.add(SimpleRNN(n_neural))
    else:
        raise ValueError('Invalid RNN type specified. Must be "LSTM" or "GRU".')
    model.add(Dense(1, activation='sigmoid'))
    print(model.summary())
    model.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model

  def evaluate_model(self, model, X, y) -> dict:
    """calcula, almacena y muestra diferentes métricas del modelo
      : param model: modelo a analizar
      : param X: set de datos 
      : param y: etiquetas de x
      return dict: con todas las metricas"""
    loss, accuracy = model.evaluate(X, y, verbose=0)
    y_pred = model.predict(X)
    y_pred_binary = np.where(y_pred >= 0.5, 1, 0)
    precision = precision_score(y, y_pred_binary)
    recall = recall_score(y, y_pred_binary)
    f1 = f1_score(y, y_pred_binary)
    metrics = {'loss': loss, 'accuracy': accuracy, 'precision': precision, 'recall': recall, 'f1': f1}
    
    # Imprimimos las métricas de forma legible para el usuario
    print(f"Loss: {metrics['loss']:.4f}")
    print(f"Accuracy: {metrics['accuracy']*100:.2f}%")
    print(f"Precision: {metrics['precision']*100:.2f}%")
    print(f"Recall: {metrics['recall']*100:.2f}%")
    print(f"F1-score: {metrics['f1']*100:.2f}%")
    return metrics

  def modelCnnW2VEmbedding(self,rnn_type: str,n_neural: int ):
    """Realiza el entrenamiento con la capa de embedding que se le pasa por parámetro
      : param rnn_type: metodo a aplicar en la capa profunda de la CNN
      : param n_neural: numero de neuronas a usar
      return """
    print(f'\n####################### Modelo con capa {rnn_type} ####################################\n')
    print(f'[INFO] Generando el modelo...')
    model_custom = Sequential()
    #pasamos directamente la capa que hemos generado
    model_custom.add(self.embedding_layer)
    if rnn_type == 'LSTM':
      model_custom.add(LSTM(n_neural,dropout=0.2, recurrent_dropout=0.2))
    elif rnn_type == 'GRU':
      model_custom.add(GRUV2(n_neural,dropout=0.2, recurrent_dropout=0.2))  
    elif rnn_type == 'SimpleRNN':
      model_custom.add(SimpleRNN(n_neural))
    else:
      raise ValueError('Invalid RNN type specified. Must be "LSTM" or "GRU".')
    model_custom.add(Dense(1, activation="sigmoid"))
    print(model_custom.summary())
    model_custom.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])
    return model_custom

    


In [ ]:
#Creamos la instancia a la clase
cnn_training = TrainingCNN(padded_sequences_train,padded_sequences_test, y_train, y_test ,embedding_layer)
#Llamamos a la funcion run para desarrollar los entrenamientos y generar un df con las metricas de los 3
metrics_cnn, metrics_cnn_custom = cnn_training.run()

In [ ]:
# Realizamos el guardado del df para su uso en el notebook de conclusiones
metrics_cnn.to_csv(f'{path}metrics_cnn.csv', index=False)
metrics_cnn